In [ ]:
# Importing the necessary libraries

import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

import tensorflow_datasets as tfds
from tensorflow.train import Checkpoint
from tensorflow_addons.layers import InstanceNormalization 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, Activation, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.initializers import RandomNormal

In [ ]:
# loading the dataset from pre defined tensorflow_dataset module (monet2photo)
data = tfds.load('cycle_gan/horse2zebra', as_supervised=True)

# assigning the monet to x and photo to y 
train_zebras = data['trainB']

In [ ]:
train_zebras

In [ ]:
Elephant_dir = '../DATA/elephants/'
train_elephants = tf.data.Dataset.list_files(Elephant_dir + '*', shuffle=True)

def parse_image(filename):
  parts = tf.strings.split(filename, os.sep)
  label = parts[-2]

  image = tf.io.read_file(filename)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, [256, 256])
  return image, label

train_elephants = train_elephants.map(parse_image)

In [ ]:
# Declaring the constants
AUTOTUNE = tf.data.experimental.AUTOTUNE

# used to prevent identity loss i.e. loss in the color of picture when transforming
LAMBDA = 10

# number of iteration to perform the training 
EPOCHS = 300

# used to shuffle 1000 images
BUFFER_SIZE = 1000

# declaring batches to train on a number of specified images simultaneously
BATCH_SIZE = 1

# image dimensions
IMG_WIDTH = 256
IMG_HEIGHT = 256
CHANNELS = 3

# setting random normalized weights 
weight_initializer = RandomNormal(stddev=0.02)

# the loss function used here is BinaryCrossentropy as we have to classify whether the generated image matches the original or not
loss = BinaryCrossentropy(from_logits=True)

# the optimizers are set to Adam with learning_rate as 2e-4 and beta_1 as 0.5
gen_g_optimizer = gen_f_optimizer = disc_x_optimizer = disc_y_optimizer = Adam(learning_rate=2e-4, beta_1=0.5)

In [ ]:
# to crop the image to our desired dimensions
def random_crop(image):
    image = tf.image.random_crop(image, size=[IMG_HEIGHT, IMG_WIDTH, CHANNELS])
    return image

In [ ]:
# to normalize the image to [-1, 1]
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

In [ ]:
# generating images with random properties such as fliping the image on vertical axis
def random_jitter(image):
    image = tf.image.resize(image, [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = random_crop(image)
    image = tf.image.random_flip_left_right(image)
    return image

In [ ]:
# method to preprocess the train images
def preprocess_train_image(image, label):
    image = random_jitter(image)
    image = normalize(image)
    return image

In [ ]:
def preprocess_elephant_image(image, label):
    image = random_jitter(image)
    #image = normalize(image)
    return image

In [ ]:
train_zebras = train_zebras.map(preprocess_train_image, num_parallel_calls=AUTOTUNE).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

train_elephants = train_elephants.map(preprocess_elephant_image, num_parallel_calls=AUTOTUNE).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# “Ck denotes a 4 × 4 Convolution-InstanceNorm-LeakyReLU layer with k filters and stride 2"

def CK(inputs, k, use_instancenorm=True):
    block = Conv2D(k, kernel_size=(4,4), strides=2, padding='same', kernel_initializer=weight_initializer)(inputs)
    if use_instancenorm:
        block = InstanceNormalization(axis=-1)(block)
    block = LeakyReLU(0.2)(block)
    return block

In [ ]:
# Method to create discriminator with the architecture mentioned in original paper (C64, C128, C256, C512)
def discriminator():

    # declaring the shape of input image (here is it 256x256x3)
    dis_input = Input(shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))
    
    # creating the arch
    d = CK(dis_input, 64, use_instancenorm=False)
    d = CK(d, 128)
    d = CK(d, 256)
    d = CK(d, 512)

    # layer to classify between the originality of generated images
    d = Conv2D(1, kernel_size=(4,4), kernel_initializer=weight_initializer, padding='same')(d)

    # generating a model using inputs and outputs
    dis = Model(dis_input, d)

    # compiling the model as it would be used to classify the images
    dis.compile(loss='mse', optimizer=disc_x_optimizer)
    return dis

    # can change the loss function to binarycrossentropy and adding a Fully connected network as ouput layer (have to experiment)

In [ ]:
# "c7s1-k denotes a 7×7 Convolution-InstanceNorm-ReLU with k filters and stride 1"
def c7s1k(inputs, k, activation):
    block = Conv2D(k, kernel_size=(7,7), strides=1, padding='same', kernel_initializer=weight_initializer)(inputs)
    block = InstanceNormalization(axis=-1)(block)
    block = Activation(activation)(block)
    return block

In [ ]:
# "dk denotes a 3×3 Convolution-InstanceNorm-ReLU with k filters and stride 2"
def dk(inputs, k):
    block = Conv2D(k, kernel_size=(3,3), strides=2, padding='same', kernel_initializer=weight_initializer)(inputs)
    block = InstanceNormalization(axis=-1)(block)
    block = Activation(tf.nn.relu)(block)
    return block

In [ ]:
# "Rk denotes a residual block that contains two 3×3 convolutional layers with k filters on each layer"
def rk(inputs, k):
    block = Conv2D(k, kernel_size=(3,3), strides=1, padding='same', kernel_initializer=weight_initializer)(inputs)
    block = InstanceNormalization(axis=-1)(block)
    block = Activation(tf.nn.relu)(block)
    block = Conv2D(k, kernel_size=(3,3), strides=1, padding='same', kernel_initializer=weight_initializer)(inputs)
    block = InstanceNormalization(axis=-1)(block)
    return block + inputs

In [ ]:
# "uk denotes a 3×3 fractional-strided-ConvolutionInstanceNorm-ReLU layer with k filters and stride ½" (Conv2d with 0.5 strides is Conv2dTranspose with 2 strides)
def uk(inputs, k):
    block = Conv2DTranspose(k, kernel_size=(3,3), strides=2, padding='same', kernel_initializer=weight_initializer)(inputs)
    block = InstanceNormalization(axis=-1)(block)
    block = Activation(tf.nn.relu)(block)
    return block

In [ ]:
# method to create the generator with arch (c7s1-64, d128, d256, R256, R256, R256, R256, R256, R256, R256, R256, R256, u128, u64, c7s1-3)

# note: we are not compiling the generator here as the only job of it is to generate the images and learn from the discriminator's response
 
def generator():
    # declaring the shape of input image (here is it 256x256x3)
    gen_inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))

    gen = c7s1k(inputs=gen_inputs, k=64, activation=tf.nn.relu)
    gen = dk(inputs=gen, k=128)
    gen = dk(inputs=gen, k=256)
    
    for _ in range(9):
        gen = rk(inputs=gen, k=256)

    gen = uk(inputs=gen, k=128)
    gen = uk(inputs=gen, k=64)
    
    gen = c7s1k(inputs=gen, k=3, activation=tf.nn.tanh)

    model = Model(gen_inputs, gen)

    return model

In [ ]:
# Defining the generators and discriminators

generator_g = generator()
generator_f = generator()

discriminator_x = discriminator()
discriminator_y = discriminator()

In [ ]:
# Methods to find the error (loss) in data

# This method is used to find how much does the generated image differ than the original 
def discriminator_loss(real, generated):
    real_loss = loss(tf.ones_like(real), real)
    generated_loss = loss(tf.zeros_like(generated), generated)
    total_loss = real_loss + generated_loss
    return total_loss * 0.5

# This method is used to find the difference between generated images to true +ve 
def generator_loss(generated):
    return loss(tf.ones_like(generated), generated)

# It is the main component in CycleGAN which finds the difference between the cycled images i.e. the image generated again from the regenerated image to find out whether the input image is again generated when used in backward
def cycle_loss(real_image, cycled_image):
    loss = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return LAMBDA * loss

# used to prevent the loss in colors and fine details
def identity_loss(real_image, sample_image):
    loss = tf.reduce_mean(tf.abs(real_image - sample_image))
    return LAMBDA * loss * 0.5

In [ ]:
# Declaring A Checkpoint so that we can start the training from where we stopped last time as we don't want to train from the scratch everytime

# path to store the checkpoints
checkpoint_path = "../Logs/checkpoints/elephant2zebra_256"

ckpt = Checkpoint(generator_g=generator_g,
                  generator_f=generator_f,
                  discriminator_x=discriminator_x,
                  discriminator_y=discriminator_y,
                  gen_g_optimizer=gen_g_optimizer,
                  gen_f_optimizer=gen_f_optimizer,
            disc_x_optimizer=disc_x_optimizer,
            disc_y_optimizer=disc_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

In [ ]:
# A tf.function() to compile the method before running it.

@tf.function
def train_step(real_x, real_y):
    # persistent is set to True because the tape is used more than
    # once to calculate the gradients.
    with tf.GradientTape(persistent=True) as tape:
        # Generator G translates X -> Y
        # Generator F translates Y -> X.

        fake_y = generator_g(real_x, training=True)
        cycled_x = generator_f(fake_y, training=True)

        fake_x = generator_f(real_y, training=True)
        cycled_y = generator_g(fake_x, training=True)

        # same_x and same_y are used for identity loss.
        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)

        disc_real_x = discriminator_x(real_x, training=True)
        disc_real_y = discriminator_y(real_y, training=True)

        disc_fake_x = discriminator_x(fake_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)

        # calculate the loss
        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)

        total_cycle_loss = cycle_loss(real_x, cycled_x) + cycle_loss(real_y, cycled_y)

        # Total generator loss = adversarial loss + cycle loss
        total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
        total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
    
    # Calculate the gradients for generator and discriminator
    generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                            generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                            generator_f.trainable_variables)
    
    discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                                discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                                discriminator_y.trainable_variables)
    
    # Apply the gradients to the optimizer
    gen_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                                generator_g.trainable_variables))
    
    gen_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                                generator_f.trainable_variables))
    
    disc_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                    discriminator_x.trainable_variables))
    
    disc_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                    discriminator_y.trainable_variables))

In [ ]:
# A method to generate a sample image on which we can see the training efficiency visually

def generate_images(model, test_image):
    predicted = model(test_image)

    plt.figure(figsize=(12,12))
    display_list = [test_image[0], predicted[0]]
    title = ['Test Image', 'Predicted Image']

    for i in range(len(display_list)):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')

    plt.show()

In [ ]:
# a samples of monet and photo to display for showcasing the training efficiency
sample_elephant = next(iter(train_elephants))
sample_zebra = next(iter(train_zebras))

plt.subplot(121)
plt.title('Photo')
plt.imshow(sample_elephant[0] * 0.5 + 0.5)
plt.subplot(122)
plt.title('Photo with random jitter')
plt.imshow(random_jitter(sample_elephant[0]) * 0.5 + 0.5)

In [ ]:
# Loop to train the GAN

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_elephants, train_zebras)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      # used to show the progress
      print ('.', end='')
    n+=1

  clear_output(wait=True)
  # Using a consistent image (sample_elephants) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_elephant)

  # defining the interval to save checkpoints
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))